In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.formula.api import ols
import matplotlib.pyplot as plt
import seaborn as sns
import math
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.dummy import DummyRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
import scipy.stats as stats
from sklearn.feature_selection import RFE

from xgboost import XGBClassifier

from mpl_toolkits.mplot3d import Axes3D
from IPython.display import Image
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, recall_score, precision_score
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import make_column_selector, make_column_transformer
from sklearn.metrics import plot_confusion_matrix
from imblearn.over_sampling import SMOTE
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import string
from nltk.corpus import stopwords
from nltk import word_tokenize
import scipy

import types
import re
import string
import nltk
import xgboost

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import KFold, cross_val_score
from sklearn.model_selection import train_test_split, GridSearchCV

import sys

from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.svm import SVC
from sklearn.compose import ColumnTransformer
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, roc_auc_score, plot_confusion_matrix

from keras.preprocessing import text, sequence
from textblob import TextBlob, Word
from wordcloud import WordCloud
import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv("../../../data/fps_main.csv")
df.head(3)
#X = df.drop(col, axis=1)
#y = df[["voted_up"]]
#X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=57, test_size = 0.5)


,steamid,appid,app_title,app_tags,review,fps,voted_up
0,76561198865552498,730,Counter-Strike: Global Offensive,"['FPS', 'Shooter', 'Multiplayer', 'Competitive...",very good game,True,1
1,76561197964533061,730,Counter-Strike: Global Offensive,"['FPS', 'Shooter', 'Multiplayer', 'Competitive...",beause S,True,1
2,76561198290998839,730,Counter-Strike: Global Offensive,"['FPS', 'Shooter', 'Multiplayer', 'Competitive...","Russians everywhere, do not recommend",True,1


In [3]:
#df = pd.concat([X_train, y_train], axis=1)
#df_test = pd.concat([X_test, y_test], axis=1)

In [4]:
#df_test.info()
def ScoreModel(model, X, y):
    preds = model.predict(X)
    acc = accuracy_score(y, preds)
    f1 = f1_score(y, preds)
    recall = recall_score(y, preds)
    precision = precision_score(y, preds)
    rockout = roc_auc_score(y, preds)
    
    print("Accuracy:  ", acc)
    print("F1 Score:  ", f1)
    print("Recall:    ", recall)
    print("Precision: ", precision)
    print("ROC_AUC:   ", rockout)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19680 entries, 0 to 19679
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   steamid    19680 non-null  int64 
 1   appid      19680 non-null  int64 
 2   app_title  19680 non-null  object
 3   app_tags   19680 non-null  object
 4   review     19680 non-null  object
 5   fps        19680 non-null  bool  
 6   voted_up   19680 non-null  int64 
dtypes: bool(1), int64(3), object(3)
memory usage: 941.8+ KB


In [6]:
#cols = ["review_id", "year", "title"]

#df = df.drop(cols, axis = 1)
#df_test= df_test.drop(cols, axis = 1)


In [7]:
df["voted_up"].value_counts(normalize=True)

0    0.515091
1    0.484909
Name: voted_up, dtype: float64

In [8]:
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/elliott/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/elliott/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [9]:
def CleanText(reviews):
    stopwords = nltk.corpus.stopwords.words('english')
    punct = string.punctuation
    lemma = nltk.WordNetLemmatizer()
    
    reviews = "".join([word for word in reviews if word not in string.punctuation])
    tokens = re.split('\W+', reviews)
    reviews = [lemma.lemmatize(word) for word in tokens if word not in stopwords]
    
# use function like this in your notebook
# df['cleaned_text'] = df['text'].apply(lambda x: CleanText(x.lower()))
    
    return reviews


In [10]:
def clean_reviews(reviews):
    reviews = "".join([word for word in reviews if word not in string.punctuation])
    tokens = re.split('\W+', reviews)
    reviews = [lemma.lemmatize(word) for word in tokens if word not in stopwords]
    return reviews

In [11]:
df['clean_review'] = df['review'].apply(lambda x: CleanText(x.lower()))

df.head()

,steamid,appid,app_title,app_tags,review,fps,voted_up,clean_review
0,76561198865552498,730,Counter-Strike: Global Offensive,"['FPS', 'Shooter', 'Multiplayer', 'Competitive...",very good game,True,1,"[good, game]"
1,76561197964533061,730,Counter-Strike: Global Offensive,"['FPS', 'Shooter', 'Multiplayer', 'Competitive...",beause S,True,1,"[, beause]"
2,76561198290998839,730,Counter-Strike: Global Offensive,"['FPS', 'Shooter', 'Multiplayer', 'Competitive...","Russians everywhere, do not recommend",True,1,"[russian, everywhere, recommend]"
3,76561198073021168,60,Ricochet,"['Action', 'FPS', 'Multiplayer', 'Classic', 'F...","best game, best game, 10/10 i r8 8/8",True,1,"[best, game, best, game, 1010, r8, 88]"
4,76561198061142423,550,Left 4 Dead 2,"['Zombies', 'Co-op', 'FPS', 'Multiplayer', 'Sh...",køb hvis du kan lide zombie spil ;D,True,1,"[køb, hvis, du, kan, lide, zombie, spil]"


In [12]:
#df['clean_review'] = df['review'].apply(lambda x: clean_reviews(x.lower()))

#df.head(10)

In [13]:
#col = ["voted_up"]
#X = df.drop(col, axis=1)
#y = df[["voted_up"]]

#X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=57, test_size = 0.54)

In [14]:
#new_df = pd.concat([X_train, y_train], axis=1)
#new_df.head(10)

In [15]:
# = new_df.loc[new_df['voted_up'] == 0]
#df_up = new_df.loc[new_df['voted_up'] == 1]

In [16]:
#wordcloud = WordCloud(max_words=100, width=400, height=200).generate(str(df_up["clean_review"]))
#plt.imshow(wordcloud, interpolation='bilinear')
#plt.axis('off')
#plt.figure(figsize=(20,10))
#plt.show()

In [17]:
#wordcloud = WordCloud(max_words=100, width=400, height=200).generate(str(df_down["clean_review"]))
#plt.imshow(wordcloud, interpolation='bilinear')
#plt.axis('off')
#plt.figure(figsize=(20,10))
#plt.show()

In [18]:
from sklearn.naive_bayes import MultinomialNB

In [19]:
vector = TfidfVectorizer(analyzer=CleanText, ngram_range=(2, 2))
X = vector.fit_transform(df["review"])

X_df = pd.DataFrame(X.toarray())
X_df.columns = vector.get_feature_names()
X_df.head(5)

,,0,00,000,00000,00001,001,00110,001500,001604,...,𝓲𝓼,𝓸𝓯,𝓻𝓾𝓼𝓼𝓲𝓪𝓷𝓼,𝓽𝓸𝔁𝓲𝓬,𝘎𝘳𝘦𝘢𝘵,𝘣𝘺,𝘤𝘩𝘦𝘢𝘵𝘦𝘳𝘴,𝘨𝘢𝘮𝘦,𝘳𝘶𝘪𝘯𝘦𝘥,𝙴ѕєαAccσυηт
0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.248761,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [20]:
#X = vector.fit_transform(df['review'])
y = df[["voted_up"]]

X_train, X_test, y_train, y_test = train_test_split(X_df, y, random_state=57, test_size = 0.5,stratify=y)

In [21]:
clf = MultinomialNB()
clf.fit(X_train, y_train)
ScoreModel(clf, X_train, y_train)

Accuracy:   0.8901422764227642
F1 Score:   0.8800621324753134
Recall:     0.8312722699643681
Precision:  0.9349363507779349
ROC_AUC:    0.8884118303856166


In [22]:
ScoreModel(clf, X_test, y_test)

Accuracy:   0.8015243902439024
F1 Score:   0.7827827827827828
Recall:     0.7374266554903605
Precision:  0.8340839061388955
ROC_AUC:    0.7996525542645172


In [23]:
#y = df["user_suggestion"]

#X_train, X_test, y_train, y_test = train_test_split(X_df, y, random_state=57)

baseline = LogisticRegression()
baseline.fit(X_train, y_train)
ScoreModel(baseline, X_train, y_train)

Accuracy:   0.9070121951219512
F1 Score:   0.9029795355741703
Recall:     0.8924753720394047
Precision:  0.9137339055793992
ROC_AUC:    0.9065848945420935


In [24]:
#vector = TfidfVectorizer(analyzer=clean_reviews, ngram_range=(2, 2))
#X = vector.fit_transform(df_test["review"])

#X_df_test = pd.DataFrame(X.toarray())
#X_df_test.columns = vector.get_feature_names()
#X_df_test.head(3)

ScoreModel(baseline, X_test, y_test)

Accuracy:   0.8128048780487804
F1 Score:   0.7982475355969332
Recall:     0.7636211232187762
Precision:  0.8361633776961909
ROC_AUC:    0.8113685726591118


In [25]:
param_grid = {'alpha': [.05,0.5, 1, 1.5, 2],
        }

In [26]:
clf = MultinomialNB()
clf.fit(X_train, y_train)


clf_gs = GridSearchCV(estimator=clf, param_grid=param_grid, scoring='accuracy')
clf_gs.fit(X_train, y_train)


GridSearchCV(estimator=MultinomialNB(),
             param_grid={'alpha': [0.05, 0.5, 1, 1.5, 2]}, scoring='accuracy')

In [27]:

best_model = clf_gs.best_estimator_

In [28]:

clf_gs.best_params_

{'alpha': 0.05}

In [29]:
ScoreModel(clf_gs, X_train, y_train)

Accuracy:   0.9341463414634147
F1 Score:   0.9316888045540797
Recall:     0.9262209180465312
Precision:  0.9372216330858961
ROC_AUC:    0.9339133787312947


In [30]:
ScoreModel(clf_gs, X_test, y_test)

Accuracy:   0.8107723577235773
F1 Score:   0.8039174389216511
Recall:     0.7998742665549036
Precision:  0.8080016934801016
ROC_AUC:    0.8104541024960784


In [32]:
boost_grid = XGBClassifier(random_state=57, objective="reg:logistic", use_label_encoder=False)

param_grid = {'n_estimators': [50, 75, 100],
                'max_depth': [1, 3, 5, 7, 10],
                'min_samples_leaf': [1, 3, 5, 7]}

gridsearch = GridSearchCV(boost_grid, param_grid, scoring="accuracy", n_jobs= -1)
gridsearch.fit(X_train, y_train)

KeyboardInterrupt: 

In [ ]:
best_model2 = clf_gs.best_estimator_

In [ ]:
clf_gs.best_params_

In [ ]:
ScoreModel(gridsearch, X_train, y_train)

In [ ]:
ScoreModel(gridsearch, X_test, y_test)